문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)

In [6]:
from typing import TypedDict, Literal, List
from langgraph.graph import StateGraph, START, END
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.documents import Document
import random

class RecommendationState(TypedDict):
    customer_preference: str
    recommended_menu: str
    menu_details: str
    recommendation_reason: str

embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings_model,
    allow_dangerous_deserialization=True
)

def get_customer_preference(state: RecommendationState) -> RecommendationState:
    preferences = ["단맛", "쓴맛", "신맛"]
    preference = random.choice(preferences)
    return {"customer_preference": preference}

def recommend_docs(docs):
    selected_doc = random.choice(docs)
    menu_name = selected_doc.metadata.get('menu_name', 'Unknown')
    menu_content = selected_doc.page_content
    reason = f"{menu_name}를 추천합니다."
    return {
        "recommended_menu": menu_name,
        "menu_details": menu_content,
        "recommendation_reason": reason
    }
def recommend_sweet_menu(state: RecommendationState) -> RecommendationState:
    sweet_queries = ["바닐라", "카라멜", "달콤한", "티라미수"]
    query = random.choice(sweet_queries)
    docs = menu_db.similarity_search(query, k=2)
    
    if docs:
        return recommend_docs(docs)
    return {
        "recommended_menu": "바닐라 라떼",
        "menu_details": "달콤한 바닐라 시럽이 들어간 라떼",
        "recommendation_reason": "달콤한 맛을 선호하는 고객님께 추천드립니다."
    }

def recommend_bitter_menu(state: RecommendationState) -> RecommendationState:
    bitter_queries = ["아메리카노", "콜드브루", "에스프레소", "원두"]
    query = random.choice(bitter_queries)
    docs = menu_db.similarity_search(query, k=2)
    
    if docs:
        return recommend_docs(docs)

    return {
        "recommended_menu": "아메리카노",
        "menu_details": "진한 에스프레소에 뜨거운 물을 더한 클래식 커피",
        "recommendation_reason": "진한 커피 맛을 선호하는 고객님께 추천드립니다."
    }

def recommend_sour_menu(state: RecommendationState) -> RecommendationState:
    sour_queries = ["녹차", "과일", "상큼한", "프라푸치노"]
    query = random.choice(sour_queries)
    docs = menu_db.similarity_search(query, k=2)
    if docs:
        return recommend_docs(docs)
    
    return {
        "recommended_menu": "녹차 라떼",
        "menu_details": "말차 파우더와 스팀 밀크로 만든 건강한 음료",
        "recommendation_reason": "상큼한 맛을 선호하는 고객님께 추천드립니다."
    }

def decide_recommendation_path(state: RecommendationState) -> Literal["sweet", "bitter", "sour"]:
    preference = state["customer_preference"]
    if preference == "단맛":
        return "sweet"
    elif preference == "쓴맛":
        return "bitter"
    else:
        return "sour"

# 3단계: 그래프 구성
builder = StateGraph(RecommendationState)

builder.add_node("get_preference", get_customer_preference)
builder.add_node("recommend_sweet", recommend_sweet_menu)
builder.add_node("recommend_bitter", recommend_bitter_menu)
builder.add_node("recommend_sour", recommend_sour_menu)

builder.add_edge(START, "get_preference")
builder.add_conditional_edges(
    "get_preference",
    decide_recommendation_path,
    {
        "sweet": "recommend_sweet",
        "bitter": "recommend_bitter",
        "sour": "recommend_sour"
    }
)
builder.add_edge("recommend_sweet", END)
builder.add_edge("recommend_bitter", END)
builder.add_edge("recommend_sour", END)

graph = builder.compile()

# 4단계: 실행 및 테스트
initial_state = {
    "customer_preference": "",
    "recommended_menu": "",
    "menu_details": "",
    "recommendation_reason": ""
}

result = graph.invoke(initial_state)
print("\n=== 최종 추천 결과 ===")
print(f"고객 취향: {result['customer_preference']}")
print(f"추천 메뉴: {result['recommended_menu']}")
print(f"메뉴 상세: {result['menu_details']}")
print(f"추천 이유: {result['recommendation_reason']}")


=== 최종 추천 결과 ===
고객 취향: 신맛
추천 메뉴: Unknown
메뉴 상세: 4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.

5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.
추천 이유: Unknown를 추천합니다.
